In [1]:
import sys
sys.path.append('../')
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from src.eda_methods import show_barplot, load_data, load_data_all
from pandas.plotting import register_matplotlib_converters

pd.set_option('display.max_colwidth', 125)
register_matplotlib_converters()

In [2]:
# old_commits, new_commits, cc_df, bic = load_data('accumulo')
old_commits, new_commits, cc_df, bic = load_data_all(nrows=300)

old_commits = old_commits.droplevel(0).sort_index()
new_commits = new_commits.droplevel(0).sort_index()
cc_df = cc_df.droplevel(0).sort_index().reset_index(drop=True)
bic = bic.droplevel(0).sort_index().reset_index(drop=True)

In [0]:
# selects cochange of at least two components with confidence higher that 0.5
relevant_cc =\
    cc_df[
        (cc_df.support_count > 1) & (cc_df.confidence >= 0.5)
    ].reset_index(drop=True).copy()

# relevant_cc = cc_df.reset_index(drop=True).copy()

# frees memory
del cc_df

**Note** that the hashes for cochange are the same as in new_commits but the hashes for bug introducing commits (bic) is the same as in old_commits. Thus, we must convert bic hashes into new_commits hashes.

In [0]:
# relevant_cc.iloc[0].commit_hash[0] in new_commits.values

# drops duplicated hashes 
bic = bic[~bic.duplicated()].reset_index(drop=True)
print('Number of bug introducing commits: {0}'.format(len(bic)))

In [0]:
# creates a dataframe to support conversion of old hashes into new ones
conv_df = pd.DataFrame()
conv_df['old_hash'] = old_commits.sort_index()
conv_df['new_hash'] =\
    new_commits.sort_index()\
    .reset_index(drop=True)[
        old_commits\
        .sort_index()\
        .reset_index(drop=True)\
        .index
    ].values

In [0]:
# converts bic's old hashes into new ones
bic =\
    conv_df.set_index('old_hash')\
    .squeeze()[bic.values]\
    .reset_index(drop=True)\
    .fillna('thereAreNoHashesHereDueToSmallSampleSize')

# frees memory
del conv_df

### > How frequent are commits that introduce bugs?

In [0]:
# len(np.unique(bic.values)) / len(np.unique(old_commits.values))
len(np.intersect1d(bic.values, new_commits.values)) / len(np.unique(new_commits.values))

### > How common is cochange in the project?

In [0]:
print(
    '{0:.3f} of all commits show cochange dependency between components (support > 1, confidence >= 0.5).'
    .format(
        len(np.unique(np.hstack(relevant_cc.commit_hash.values))) / len(np.unique(old_commits.values))
    )
)

### > Of all commits with cochange dependency, how many introduce bugs?

In [0]:
len(np.intersect1d(np.hstack(relevant_cc.commit_hash.values), bic.values)) / len(np.unique(np.hstack(relevant_cc.commit_hash.values)))

### > Of all commits that introduce bugs, how many have cochange dependency? (questionable)

In [0]:
len(np.intersect1d(np.hstack(relevant_cc.commit_hash.values), bic.values)) / len(bic.values)

## Distribuition of bug introducing commits over cochange dependency support (level)

### > Broad Overview

In [0]:
show_barplot(relevant_cc, 'support_count', y_logscale='log')

### > How many of the observations are unique?

In [0]:
# hashes the list of commits to check for duplicates
relevant_cc['id_hash'] = relevant_cc.commit_hash.apply(lambda x: hash(tuple(x)))

In [0]:
print(
    'Unique hashes of list of commit_hash: {0}\n'
    .format(
        len(relevant_cc) - relevant_cc.set_index('id_hash').index.duplicated().sum()
    )
    + 'Should the order of components influence the co-change dependency (a,b != b,a ?)? It does at this time.'
)

In [0]:
unique_cc =\
    relevant_cc[
        ~relevant_cc.set_index('id_hash')
        .index.duplicated()
    ].reset_index(drop=True).copy()

unique_cc

# frees memory
del relevant_cc

In [0]:
show_barplot(unique_cc, 'support_count', y_logscale=True)

### > How are bugs distributed over support_count categories?

In [0]:
unique_cc['bic_count'] =\
    unique_cc['commit_hash'].apply(
        lambda x: len(
            [
                commit_h for commit_h in x
                if commit_h in bic.values
            ]
        )
    )

try:
    unique_cc['bic_percent'] =\
        unique_cc[unique_cc['bic_count'] > 0][['commit_hash', 'bic_count']]\
        .apply(lambda x: x[1] / len(x[0]), axis=1)


    unique_cc['bic_percent'] = unique_cc['bic_percent'].fillna(0.0)

except ValueError:
    unique_cc['bic_percent'] = unique_cc['commit_hash'].apply(lambda x: 0.0)

In [0]:
for name, group in unique_cc.groupby('support_count'):
    print('> Support count ', name)
    print('Cochange instances: \t\t', len(group))
    print('Cochange instances with bugs: \t', group[group['bic_count'] > 0]['bic_count'].count())
    print('Individual bic count: \t\t', group['bic_count'].sum(), '\n')

In [0]:
# sns.catplot();
# unique_cc.groupby('support_count').agg(lambda x: x[x.bic_count > 0].bic_count.count() / len(x)).bic_count
compare_cc_bugs = pd.DataFrame()
compare_cc_bugs['cc_bugs'] = unique_cc.groupby('support_count').agg(lambda x: x[x.bic_count > 0].count()).bic_count
compare_cc_bugs['cc_count'] = unique_cc.groupby('support_count').count().commit_hash
compare_cc_bugs['cc_groups_with_bugs_percent'] = compare_cc_bugs.cc_bugs / compare_cc_bugs.cc_count
compare_cc_bugs['unique_commits_with_bugs_percent'] = unique_cc.groupby('support_count').mean().bic_percent
compare_cc_bugs['unique_commits_with_bugs_std'] = unique_cc.groupby('support_count').bic_percent.std().fillna(0.0)

In [0]:
sns.set_style("whitegrid", {'axes.grid' : False})

# creates the subplot
fig, ax1 = plt.subplots(figsize=(20, 8))
width = 0.35

# sets up the primary plot
ax1.set_title('Cochange (CC) Dependency Analysis', fontsize=14)
ax1.set_ylabel('Count')
ax1.set_xlabel('Support Count')
ax1.spines['top'].set_visible(False)
ax1.spines['right'].set_visible(False)
ax1 .spines['left'].set_visible(False)


# plots the first bar
bar1 = ax1.bar(
    compare_cc_bugs.reset_index(drop=True).index + 1 - width/2,
    compare_cc_bugs['cc_count'],
    width,
    log=True,
    label='CC groups'
)

# plots the second bar
try:
    bar2 = ax1.bar(
        compare_cc_bugs.reset_index(drop=True).index + 1 + width/2,
        compare_cc_bugs['cc_bugs'],
        width,
        log=True,
        label='CC groups with bugs'
    )
    plt.legend(loc='upper left', bbox_to_anchor=(0.1, 1.0))

except ValueError:
    pass

# sets up the secondary subplot
ax2 = ax1.twinx()
# ax2.spines['top'].set_visible(False)
# ax2.spines['right'].set_visible(False)
# ax2.spines['left'].set_visible(False)
ax2.set_ylabel('Percentage')

try:
    # plots the first line
    line1 = ax2.plot(
        compare_cc_bugs.index,
        compare_cc_bugs['cc_groups_with_bugs_percent'],
        label='Percentage of CC groups with bugs',
        color='b'
    )

except ValueError:
    pass

try:
    # plots the second line
    line2 = ax2.plot(
        compare_cc_bugs.index,
        compare_cc_bugs['unique_commits_with_bugs_percent'],
        label='Mean percentage of commits with bugs within CC groups',
        color='orange'
    )
except ValueError:
    pass

try:
    # plots the standard deviation area
    fill = ax2.fill_between(
        compare_cc_bugs.index,
        (compare_cc_bugs['unique_commits_with_bugs_percent'] + 2 * compare_cc_bugs['unique_commits_with_bugs_std']).values,
        y2=(compare_cc_bugs['unique_commits_with_bugs_percent'] - 2 * compare_cc_bugs['unique_commits_with_bugs_std']).apply(lambda x: 0 if x < 0 else x).values,
        color='orange', alpha=.1, label='Standard deviation of the mean percentage'
    )
    plt.legend(loc='upper right', bbox_to_anchor=(0.95, 0.95))

except ValueError:
    pass

plt.show()

Compare the same data with commits without cochange. Relate the percentage of bugs in commits without cochange and commits with cochange

In [0]:
# new_commits[new_commits.apply(lambda x: x in cochange_commits)]

# gets an array of unique commit hashes with cochange dependency
cochange_commits = np.unique(np.hstack(unique_cc['commit_hash'].values))

# filters the series of commits to have only cochange dependency
if type(cochange_commits) == np.ndarray:
    cochange_commits = new_commits[new_commits.apply(lambda x: x in cochange_commits)]

In [0]:
# initializes dataframes for plotting
commits_monthly_count = new_commits.resample('M').count()
bugs_commits_monthly_count= new_commits[new_commits.apply(lambda x: x in bic.values)].resample('M').count()
cochange_commits_monthly_count = cochange_commits.resample('M').count()
cochange_bug_commits_monthly_count = cochange_commits[cochange_commits.apply(lambda x: x in bic.values)].resample('M').count()

# creates the subplots
fig, ax = plt.subplots(2, 1, figsize=(20,5), sharex=True)
fig.text(0.5, 0.04, 'Date', ha='center', va='center')
fig.text(0.075, 0.5, 'Commit Count', ha='center', va='center', rotation='vertical')
fig.text(0.95, 0.5, 'Percentage of Bugs', ha='center', va='center', rotation='vertical')

# sets up the first plot
ax[0].set_title('Total Commits', fontsize=12)
ax[0].spines['top'].set_visible(False)
ax[0].spines['right'].set_visible(False)
ax[0].spines['left'].set_visible(False)

line1 = ax[0].plot(
    commits_monthly_count.index.date,
    commits_monthly_count.values,
    label='Monthly-aggregate commits',
    color='lightblue'
)


# line2 = ax[0].plot(
#     bugs_commits_monthly_count.index.date,
#     bugs_commits_monthly_count,
#     label='Monthly-aggregate commits with bugs',
#     color='#FF9AA2',
#     linewidth=2.0
# )
ax[0].legend(loc='upper left', bbox_to_anchor=(0.01, 1.5))

# ax2 = ax[0].twinx()
# ax2.spines['top'].set_visible(False)
# ax2.spines['right'].set_visible(False)
# ax2.spines['left'].set_visible(False)

try:
    agg_ratio = (bugs_commits_monthly_count / commits_monthly_count).fillna(0.0)
    fill = ax[0].fill_between(
        commits_monthly_count.index.date,
        (commits_monthly_count * agg_ratio).fillna(0.0).values,
        # y2=(bugs_commits_monthly_count - 100*ratio).fillna(0.0).values,
        color='#FF9AA2', alpha=.9, label='Width as percentage of bugs'
    )
    # fill = ax2.bar(ratio.index.date, ratio.values, 5.0, color='red', alpha=.5, label='Monthly Percentage of Bugs')
    ax[0].legend(loc='upper right', bbox_to_anchor=(1.0, 1.5))

except ZeroDivisionError:
    pass

# sets up the second plot
ax[1].set_title('Commits with Cochange Dependency', fontsize=12)
ax[1].spines['top'].set_visible(False)
ax[1].spines['right'].set_visible(False)
ax[1].spines['left'].set_visible(False)

line3 = ax[1].plot(
    cochange_commits_monthly_count.index.date,
    cochange_commits_monthly_count.values,
    label='Monthly-aggregate commits with cochange dependency',
    color='lightblue',
)

# line4 = ax[1].plot(
#     cochange_bug_commits_monthly_count.index.date,
#     cochange_bug_commits_monthly_count.values,
#     label='Monthly-aggregate commits with cochange dependency and bugs',
#     color='#B5EAD7',
#     linewidth=2.0
# )
ax[1].legend(loc='lower left', bbox_to_anchor=(0.01, -0.7))

# ax3 = ax[1].twinx()
# # ax3.set_ylabel('Percentage of Bugs')
# ax3.spines['top'].set_visible(False)
# ax3.spines['right'].set_visible(False)
# ax3.spines['left'].set_visible(False)

# ratio = (cochange_bug_commits_monthly_count / cochange_commits_monthly_count).fillna(0.0)
# fill = ax3.bar(ratio.index.date, ratio.values, 7.0, color='red', alpha=.5, label='Monthly Percentage of Bugs')
try:
    cc_ratio = (cochange_bug_commits_monthly_count / cochange_commits_monthly_count).fillna(0.0)
    fill = ax[1].fill_between(
        cochange_commits_monthly_count.index.date,
        (cochange_commits_monthly_count * cc_ratio).fillna(0.0).values,
        # y2=(bugs_commits_monthly_count - 100*ratio).fillna(0.0).values,
        color='#FF9AA2', alpha=.9, label='Width as percentage of bugs'
    )

except ZeroDivisionError:
    pass

plt.show()
# new_commits.resample('M').count().plot(kind='bar', figsize=(20,5))

## Normality Test

In [0]:
fig, ax = plt.subplots(3, 1, sharex=True, figsize=(20,8),)

try:
    cc_ratio.plot(
        ax=ax[0],
        kind='hist',
        label='Cochange',
        weights=(np.ones_like(cc_ratio.values) / len(cc_ratio.values))
    )

except NameError:
    pass

try:
    agg_ratio.plot(
        ax=ax[0],
        kind='hist',
        label='Aggregate',
        color='orange',
        weights=(np.ones_like(agg_ratio.values) / len(agg_ratio.values))
    )
    ax[0].set_title('Probability Density Function (PDF)', fontsize=14)
    ax[0].set_ylabel('')
    ax[0].legend(loc='upper right')

except NameError:
    pass
# --------------------------------------------------------------------

try:
    cc_ratio.plot(
        ax=ax[1],
        kind='hist',
        label='Cochange',
        density=1,
        cumulative=True
    )
    ax[1].set_title('Cumulative Distribution Function (CDF)', fontsize=14)
    ax[1].legend(loc='upper left')
    ax[1].set_ylabel('')

except NameError:
    pass
# --------------------------------------------------------------------

try:
    agg_ratio.plot(
        ax=ax[2],
        kind='hist',
        label='Aggregate',
        density=1,
        cumulative=True,
        color='orange'
    )
    ax[2].legend(loc='upper left')
    ax[2].set_ylabel('')
    ax[2].set_xlabel('Percentage of Bugs')

except NameError:
    pass

plt.show()

In [0]:
from statsmodels.graphics.gofplots import qqplot

fig, ax = plt.subplots(1, 2, figsize=(15,5), sharey=True)

try:
    qqplot(agg_ratio, line='s', ax=ax[0], label='Aggregate').gca().lines
    qqplot(cc_ratio, line='s', ax=ax[1], label='Cochange').gca().lines

    ax[0].legend(loc='upper left')
    ax[1].legend(loc='upper left')

except NameError:
    pass

Both distributions are far from normal; thus we cannot apply paired t-tests.<br />
Perhaps bootstrapping?